In [6]:
import numpy as np
import pandas as pd

import timeit

import twint
import nest_asyncio
nest_asyncio.apply()

import matplotlib
%matplotlib inline

In [4]:
def twint_search(search, username=None, since=None, until=None, limit=None):
    c = twint.Config()
    c.Search = search
    c.Username = username
    c.Since = since
    c.Until = until
    c.Limit = limit
    c.Pandas = True
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    return df

In [5]:
mask_df = twint_search(search='mask', username=None, since='2020-02-01', until='2020-03-01', limit=1000)

1233904778490589184 2020-02-29 18:59:58 EDT <Psychic_Driving> Man in the Iron Mask
1233904768268914689 2020-02-29 18:59:56 EDT <stan_khj> Good Morning Kim Hyun-Joo @kim_hyunjoo Please take care and wear your mask always!   #IHaveALover https://twitter.com/LIPS_kjs/status/1232600384020963328 …
1233904765647519744 2020-02-29 18:59:55 EDT <ErnestMEdsel1> #BlackMonday #BraceForImpact #HeadsDown Boeing execs in corner offices tell employees: #BYOB of hand sanitizer + own mask & gloves. The $BA #Boeing plan for #CoronaVirus #COVID19 deaths, illness, slowdowns for #Seattle employees #Everett #Renton factories. https://komonews.com/news/local/hand-sanitizer-supplies-falling-dramatically-in-wake-of-coronavirus …
1233904760723517440 2020-02-29 18:59:54 EDT <crethdavis> yeah, it’s helpful to educate people away from buying masks if they’re healthy. it hurts everyone when masks are stockpiled by the healthy. NBC, CBS, ABC reporting correctly that masks don’t prevent the person wearing the mask fro

1233904299870060544 2020-02-29 18:58:04 EDT <takanomartina> Good Morning, good news an end to re-selling masks for profit, which seems to be the source of the mask shortage according to this article. Internet sites asked to stop individuals from offering masks  : The Asahi Shimbun  http://www.asahi.com/ajw/articles/13175677 …
1233904295843483648 2020-02-29 18:58:03 EDT <R0816MRYO> 座れるようになったらしくて おじさんはずっとピエロ眺めてた
1233904289694679040 2020-02-29 18:58:02 EDT <ywwwc1> So at ur point of view, if you don't have N95, you'd better wear no mask, since other masks are 'ineffective'？
1233904284649099264 2020-02-29 18:58:00 EDT <PimpAssM> I gotta go to all these different stores cause my son want a mask
1233904282891517952 2020-02-29 18:58:00 EDT <LamontMagee> Five 80's cartoons that you'd love to write as a feature... Go! 1. Robotech 2. Voltron 3. Thundercats 4. MASK (just heard this is off the market) 5. Defenders of the Earth  If you have the rights, I'm your huckleberry.
1233904257654329344 2020

1233903610058203136 2020-02-29 18:55:20 EDT <MikeAustin> As a reminder, the standard mask worn in this photo provides minimal protection against a virus. It is designed to keep droplets and illness in, not out.  https://twitter.com/cnnbrk/status/1233896047497355264 …
1233903609013821440 2020-02-29 18:55:19 EDT <AlirezaMotameni> Trump administration put an order for monthly 45 million face mask production to 3M company. امروز دولت آمریکا درخواست تولید ماهانه ۴۵ میلیون ماسک صورت را به کارخانه 3M داده است.
1233903607419768832 2020-02-29 18:55:19 EDT <nemomo_san> mask shortage in Japan...
1233903606908231682 2020-02-29 18:55:19 EDT <trishankkarthik> It’s a safe place, like China, wouldn’t be too worried   Just maybe get a mask for the way back...
1233903593138225153 2020-02-29 18:55:16 EDT <lenswagameme> nobody cared about me until I put on the mask  pic.twitter.com/VQ53rWkIpi
1233903572821008385 2020-02-29 18:55:11 EDT <EGA65> Yeah, you're either lying or your husband's an idiot. Any half

1233903176866287616 2020-02-29 18:53:36 EDT <simonvouet1> Mike Pence on Coronavirus: ‘The Average American Does Not Need to Go Out and Buy a Mask’  http://bit.ly/3aclhyQ  via @BreitbartNews
1233903171661115397 2020-02-29 18:53:35 EDT <suckJinnn> Oh god if I see anyone wearing a mask I’m turning my ass around and going home idc GABDJD
1233903164031676417 2020-02-29 18:53:33 EDT <KevinM42547559> I wonder if there was anybody there with a mask on? The media is really promoting hysteria about the coronavirus. Soon this will pass and they will have to create another story to push.
1233903152178446336 2020-02-29 18:53:30 EDT <GilliJamtin> Nivea for mature skin or anything from Manuka Doctor (the serum is amazing). My friend uses a pumpkin mask once a week and her 63yo skin is flawless.
1233903132783960065 2020-02-29 18:53:26 EDT <penichesos> Darién y Tuxedo Mask???
1233903126048104448 2020-02-29 18:53:24 EDT <luminaryunivers> " AH- oh hey sushi! "  that made kaito jump but then smile . he wa

1233902860603142145 2020-02-29 18:52:21 EDT <idy_inuk> Face mask is 15k at Prince Ebeano Supermarket. Small sanitizer bottle is 2000. Wow. Just wow.
1233902854080827392 2020-02-29 18:52:19 EDT <ScottChu666> better use ur resource to import enough mask from China & sell them at fair price. public will love it. China produce 70million daily now.
1233902839329505281 2020-02-29 18:52:16 EDT <myu_myu_melody> 兵斗さん、おはよ〜☀️ ほんとに不思議だよね😊 だから出会えたことに感謝✨兵斗さんにも感謝✨✨  今日もサディスティックに頑張るよ🤣
1233902826268569600 2020-02-29 18:52:13 EDT <mzw007> Coronavirus outbreak: When and how to wear a mask | THE BIG STORY | The ...  https://youtu.be/Zt6CbSkNgp0  via @YouTube
1233902825534623746 2020-02-29 18:52:13 EDT <heyitslizabeth> overheard in urgent care about the corona virus:  receptionist: “a mask isn’t gonna solve your problems, just wash your damn hands!”
1233902797881516033 2020-02-29 18:52:06 EDT <JPH32652648> You know, people in China were told N95 masks are necessary for coronavirus in the very early stage. 

...

1233895712405868544 2020-02-29 18:23:57 EDT <min_0314> お願いします！！ 24日行くなら妹も行きたいとのこと(MASK目当てで)
1233895703551737857 2020-02-29 18:23:55 EDT <kingseungy8n> drink more water and also don't forget to wear mask whenever you go out. stay safe and keep hygiene💙✨
1233895673860222976 2020-02-29 18:23:47 EDT <jacarandaAgapan> Face Mask for my Pooh(o^^o)  pic.twitter.com/eZd2otpkEU
1233895671343788032 2020-02-29 18:23:47 EDT <ChouYajie> If wearing a mask is useless, stay at home. China uses this method to reduce the number of infected people.
1233895655908626434 2020-02-29 18:23:43 EDT <Marybeezzz> When Pence says we don’t “need to go out & buy a mask”, go out & buy one. I wouldn’t trust someone to speak about a dangerous global virus 🦠 as if he was some a virologist. #Coronavid19
1233895651332804609 2020-02-29 18:23:42 EDT <SoopremeKommand> They have on standard disposal PPE.  I carry the same kit with me for home health.  What they lack is the higher level mask, which is in very short supply. Our 

1233894969649311744 2020-02-29 18:21:00 EDT <ViggarZ> I'm not the biggest fan of star wars but all the helmet/mask designs are fucking gorgeous
1233894950116392963 2020-02-29 18:20:55 EDT <heatherleanne> Also! I have a face mask because I’m immunocompromised, but it’s a fancy one u had to be fitted for.
1233894938917703680 2020-02-29 18:20:52 EDT <hxneywoo> You should wear a mask 🥺 Just in case JSJSJS
1233894927119060992 2020-02-29 18:20:49 EDT <azemar_vladimir> SI POUVWA PHTK TAP REGLE YON BAGAY SERYE YO TE SIPOZE KÒMANSE AP FÈ DISTRIBISYON MASK POU PREPARE VINI KOWONA VIRIS LAN
1233894922907865088 2020-02-29 18:20:48 EDT <burritojustice> “When i went to the hospital, I walked with a mask instead of using public transportation to prevent the spread of the virus.“  https://twitter.com/josungkim/status/1233219595219005440 …
1233894906298540034 2020-02-29 18:20:44 EDT <peacelove_all> It should be the N95 antivirus mask. There is a difference.
1233894903165353984 2020-02-29 18:20:44 EDT <

1233894497487974400 2020-02-29 18:19:07 EDT <hate_iron_mask> みえさん おはようございます👋😃☀  桃色ってピンクですよね😊  桃の果実ってピンクじゃないのに 何で桃色っていうんだろうと思っ ていましたが桃の花の色だったの ですね❓😄
1233894494946283520 2020-02-29 18:19:06 EDT <invaderler> should i get a stylish face mask then put a disposable right over it
1233894484599099392 2020-02-29 18:19:04 EDT <MadMikeOfficial> N95 masks are your standard 3M construction masks.  Only mask that is stopping a virus is an N99 and up.
1233894478123065345 2020-02-29 18:19:02 EDT <terriarc> The one where the dying uncle gave his ungrateful family hideous masks to wear. Each mask suited their particular personality.  When they removed them, their faces took on the shape of the mask
1233894442844614656 2020-02-29 18:18:54 EDT <jasmis0914> やつは上手い
1233894433625493504 2020-02-29 18:18:52 EDT <stargazedsan> I literally just had a mental breakdown trying to watch San's vlive again and I cant do it I love him too much I love him so much the simple tilt of his head reading comments, the way his

1233894080133959686 2020-02-29 18:17:28 EDT <small_follower> MASK OFF!  pic.twitter.com/vrDD6UKqx6
1233894069908144129 2020-02-29 18:17:25 EDT <saludybienllc> Safe Mask Mascarilla Respiratoria ¿Dónde Comprar al Mejor Precio?  https://youtu.be/4SBhTCnhh4A 
1233894066074550272 2020-02-29 18:17:24 EDT <ronniedepesa> A healthy person can have COVID-19 and have mild to not symptoms but still can pass it on to people who are not at healthy.  Older, compromised immune systems, cancers, heart problems etc etc.  Thats why I would wear a mask, to protect those others more than myself.
1233894058969518081 2020-02-29 18:17:22 EDT <KroneRandy> Coronaviris : So fear did in the stock market. Germ mask sold out every where. New cases coming. Local Walmart looks like hurricane season. Said it would take something big to bring us all together. Not ! Nothing will stop the Dem's from hating Deporables & President Trump.
1233894033614876672 2020-02-29 18:17:16 EDT <RealRowdyBob> I've discovered if I really

1233893410651738113 2020-02-29 18:14:48 EDT <fulIersun> my eyes are like almost the same color as my mask heh  pic.twitter.com/vW6fqx48V3
1233893404678877184 2020-02-29 18:14:46 EDT <_prettybrowngrl> Baby why that thang bubbling up like a face mask lmao
1233893397363974144 2020-02-29 18:14:45 EDT <ZionBuild___> Wear a mask and see how people react.  Act sick too
1233893391718637568 2020-02-29 18:14:43 EDT <whitequark> i have a 3M full mask for chemistry reasons but it seems like no one locally sells cartridges that are N95 or higher? like, even in principle, they aren't sold out??
1233893387822096384 2020-02-29 18:14:42 EDT <libbyjxde> MATE SAME THE CREEPY MASK
1233893383443230720 2020-02-29 18:14:41 EDT <arch_logan> He don’t take the mask off tho easy indicator
1233893381660725255 2020-02-29 18:14:41 EDT <alou_jop> Walla ce Badara il m’a foutu la trouille de ma life. Amena yo khamni kéne douko kafé koi. Déjà nioune danio baari waax mask meunoul dara ssi nioune. Yalla naniou ssi Yalla 

In [11]:
mask_df.keys()

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username',
       'name', 'day', 'hour', 'link', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [12]:
mask_df.head()

,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1233904778490589184,1233685012777709568,1583020798000,2020-02-29 18:59:58,EDT,,Man in the Iron Mask,[],[],1093532256,...,,,,,,"[{'user_id': '1093532256', 'username': 'Psychi...",,,,
1,1233904768268914689,1233904768268914689,1583020796000,2020-02-29 18:59:56,EDT,,Good Morning Kim Hyun-Joo @kim_hyunjoo Please ...,[#ihavealover],[],1220340939031375872,...,,,,,,"[{'user_id': '1220340939031375872', 'username'...",,,,
2,1233904765647519744,1233904765647519744,1583020795000,2020-02-29 18:59:55,EDT,,#BlackMonday #BraceForImpact #HeadsDown Boeing...,"[#blackmonday, #braceforimpact, #headsdown, #b...",[$ba],474631096,...,,,,,,"[{'user_id': '474631096', 'username': 'ErnestM...",,,,
3,1233904760723517440,1233886355064786951,1583020794000,2020-02-29 18:59:54,EDT,,"yeah, it’s helpful to educate people away from...",[],[],40953972,...,,,,,,"[{'user_id': '40953972', 'username': 'crethdav...",,,,
4,1233904717652201475,1233643387460505600,1583020784000,2020-02-29 18:59:44,EDT,,Im not seeing any politicians even put on the ...,[],[],762409496,...,,,,,,"[{'user_id': '762409496', 'username': 'Zeratot...",,,,
